In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
device=('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
import pandas as pd
df=pd.read_csv('data/iris3.csv', delimiter=',') #numpy에서 첫줄 들어오니까 한줄 skip 하고 가져오기 #get_dummies 사용

In [4]:
X=df.iloc[:, :-1]
y=df.iloc[:, -1]

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
dtypes: float64(4)
memory usage: 4.8 KB


In [6]:
X=torch.from_numpy(X.values) #df -> torch #.values 또는 to_numpy()를 활용하면 ndarray형태로 변환할 수 있다.

In [7]:
print(type(X))

<class 'torch.Tensor'>


In [8]:
y=pd.get_dummies(y)
y=torch.from_numpy(y.values)

In [9]:
print(type(y)) #df -> torch

<class 'torch.Tensor'>


In [10]:
print(y[:5])

tensor([[1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]], dtype=torch.uint8)


In [11]:
class Iris(nn.Module):
    def __init__(self):
        super(Iris, self).__init__()
        self.hidden_linear=nn.Linear(4, 12)
        self.hidden_activation=nn.ReLU()
        self.hidden_linear2=nn.Linear(12, 8)
        self.hidden_activation2=nn.ReLU()
        self.output_linear=nn.Linear(8,3)
        
    def forward(self, x):
        x=self.hidden_linear(x)
        x=self.hidden_activation(x)
        x=self.hidden_linear2(x)
        x=self.hidden_activation2(x)
        x=self.output_linear(x)
        
        return x

In [12]:
model=Iris().to(device)
model

Iris(
  (hidden_linear): Linear(in_features=4, out_features=12, bias=True)
  (hidden_activation): ReLU()
  (hidden_linear2): Linear(in_features=12, out_features=8, bias=True)
  (hidden_activation2): ReLU()
  (output_linear): Linear(in_features=8, out_features=3, bias=True)
)

In [13]:
def count_parameters(model):
    total_param = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_param = np.prod(param.size())
            if param.dim() > 1:
                print(name, ':', ' x '.join(str(x) for x in list(param.size())[::-1]), '=', num_param)
            else:
                print(name, ':', num_param)
                print('-' * 40)
            total_param += num_param
    print('total:', total_param)

In [14]:
count_parameters(model)

hidden_linear.weight : 4 x 12 = 48
hidden_linear.bias : 12
----------------------------------------
hidden_linear2.weight : 12 x 8 = 96
hidden_linear2.bias : 8
----------------------------------------
output_linear.weight : 8 x 3 = 24
output_linear.bias : 3
----------------------------------------
total: 191


In [15]:
ds=TensorDataset(X, y)
dataloader= DataLoader(ds, batch_size=5)
optimizer=optim.Adam(model.parameters())

n_epochs=100
loss_fn=nn.CrossEntropyLoss() #이 함수는 알아서 값이 들어오면 softmax로 펼쳐줌. 따라서 softmax하기 직전 값을 넣어줘야함

for epoch in range(n_epochs):
    for data, label in dataloader:
        out=model(data.type(torch.FloatTensor).to(device)) #forward propagation
        loss=loss_fn(out, label.type(torch.FloatTensor).to(device)) #이미 2차원 배열로 만들어져 있어서 unsqueeze 필요 없음
        loss.backward() #개별 파라미터 기울기 계산
        optimizer.step() #개별 파라미터 갱신
        optimizer.zero_grad() 
    print("Epoch: %d, Loss: %.4f" %(epoch, float(loss)))

Epoch: 0, Loss: 1.5171


Epoch: 1, Loss: 1.4353
Epoch: 2, Loss: 1.3937
Epoch: 3, Loss: 1.3647
Epoch: 4, Loss: 1.3312
Epoch: 5, Loss: 1.2863
Epoch: 6, Loss: 1.2301
Epoch: 7, Loss: 1.1560
Epoch: 8, Loss: 1.0691
Epoch: 9, Loss: 0.9891
Epoch: 10, Loss: 0.8893
Epoch: 11, Loss: 0.8143
Epoch: 12, Loss: 0.7389
Epoch: 13, Loss: 0.6807
Epoch: 14, Loss: 0.6339
Epoch: 15, Loss: 0.5992
Epoch: 16, Loss: 0.5739
Epoch: 17, Loss: 0.5577
Epoch: 18, Loss: 0.5486
Epoch: 19, Loss: 0.5423
Epoch: 20, Loss: 0.5364
Epoch: 21, Loss: 0.5312
Epoch: 22, Loss: 0.5277
Epoch: 23, Loss: 0.5270
Epoch: 24, Loss: 0.5229
Epoch: 25, Loss: 0.5206
Epoch: 26, Loss: 0.5151
Epoch: 27, Loss: 0.5170
Epoch: 28, Loss: 0.5103
Epoch: 29, Loss: 0.5082
Epoch: 30, Loss: 0.5012
Epoch: 31, Loss: 0.5005
Epoch: 32, Loss: 0.4898
Epoch: 33, Loss: 0.4894
Epoch: 34, Loss: 0.4797
Epoch: 35, Loss: 0.4786
Epoch: 36, Loss: 0.4683
Epoch: 37, Loss: 0.4669
Epoch: 38, Loss: 0.4559
Epoch: 39, Loss: 0.4574
Epoch: 40, Loss: 0.4432
Epoch: 41, Loss: 0.4438
Epoch: 42, Loss: 0.4323
E

In [21]:
test_loader = DataLoader(ds, batch_size=5)
correct = 0
flag = True

with torch.no_grad():
    for data, label in test_loader:
        output = model(data.type(torch.FloatTensor).to(device))
        pred = output.max(1)[1] #output을 1번째 차원으로 max하여 그 index를 추출
        target = label.max(1)[1]  #label을 1번째 차원으로 max하여 그 index를 추출
        if flag:
            print("output",output)
            print("output.max(0)", output.max(0)) #row 중에 max
            print("output.max(1)", output.max(1)) #col 중에 max
            print('-'* 30)
            print("label", label)
            print("label.max(0)", label.max(0))
            print("label.max(1)", label.max(1))
            flag = False
        correct += pred.eq(target.to(device)).sum().item() #5라는 tensor의 값을 .item()으로 꺼냄
        
print('\nTest set: Accuracy: {}/{} ({:f})\n'.format(correct, len(test_loader.dataset), correct / len(test_loader.dataset)))
        
print("Accuracy: %f" % (correct / len(test_loader.dataset)))

output tensor([[  7.9645,   2.0204, -13.2779],
        [  7.1925,   1.8526, -12.0034],
        [  7.3750,   1.8940, -12.3093],
        [  7.0927,   1.8341, -11.8471],
        [  8.0330,   2.0364, -13.3939]])
output.max(0) torch.return_types.max(
values=tensor([  8.0330,   2.0364, -11.8471]),
indices=tensor([4, 4, 3]))
output.max(1) torch.return_types.max(
values=tensor([7.9645, 7.1925, 7.3750, 7.0927, 8.0330]),
indices=tensor([0, 0, 0, 0, 0]))
------------------------------
label tensor([[1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]], dtype=torch.uint8)
label.max(0) torch.return_types.max(
values=tensor([1, 0, 0], dtype=torch.uint8),
indices=tensor([0, 0, 0]))
label.max(1) torch.return_types.max(
values=tensor([1, 1, 1, 1, 1], dtype=torch.uint8),
indices=tensor([0, 0, 0, 0, 0]))

Test set: Accuracy: 146/150 (0.973333)

Accuracy: 0.973333
